# Jonathan Halverson
# October 12, 2016
# Converting an RDD to a DataFrame in Spark 2

### Converting a Spark DF to a Pandas DF

In [14]:
df = spark.createDataFrame([(["a", "b", "c"],)], ["words"])
df.show()

+---------+
|    words|
+---------+
|[a, b, c]|
+---------+



In [15]:
import pandas as pd
x = df.toPandas()
x

,words
0,"[a, b, c]"


### Adding a second row to the DF

In [16]:
df = spark.createDataFrame([(["a", "b", "c"],), (["e", "f", "g"],)], ["words"]).show()

+---------+
|    words|
+---------+
|[a, b, c]|
|[e, f, g]|
+---------+



In [17]:
df = spark.createDataFrame([(["a", "b", "c"],), (["e", "f", "g"],)], schema=["words"])
df.collect()

[Row(words=[u'a', u'b', u'c']), Row(words=[u'e', u'f', u'g'])]

### Working with vectors

In [18]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import DCT

df1 = spark.createDataFrame([(Vectors.dense([5.0, 8.0, 6.0]),)], ["vec"])
dct = DCT(inverse=False, inputCol="vec", outputCol="resultVec")
df2 = dct.transform(df1)
df2.show()

+-------------+--------------------+
|          vec|           resultVec|
+-------------+--------------------+
|[5.0,8.0,6.0]|[10.9696551146028...|
+-------------+--------------------+



### toDF

In [19]:
ranges = sc.parallelize([[12, 45], [9, 11], [31, 122], [88, 109], [17, 61]])
print type(ranges)

<class 'pyspark.rdd.RDD'>


In [20]:
df = ranges.toDF(schema=['a', 'b'])
df.show()
print type(df)
print df.printSchema()

+---+---+
|  a|  b|
+---+---+
| 12| 45|
|  9| 11|
| 31|122|
| 88|109|
| 17| 61|
+---+---+

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- a: long (nullable = true)
 |-- b: long (nullable = true)

None


### Email classifier

In [21]:
from pyspark.ml.feature import HashingTF

tf = HashingTF(numFeatures=2**18, inputCol="words", outputCol="vecs")
df = spark.createDataFrame([(['There', 'will', 'be', 'cake'],), (['I', 'will', 'run', 'again'],)], ["words"])
out = tf.transform(df)
out.show()

+--------------------+--------------------+
|               words|                vecs|
+--------------------+--------------------+
|[There, will, be,...|(262144,[13007,89...|
|[I, will, run, ag...|(262144,[89356,10...|
+--------------------+--------------------+



In [32]:
ham = sc.textFile('ham.txt')
spam = sc.textFile('spam.txt')

In [33]:
hamFeatures = ham.map(lambda email: tf.transform([email.split()]))
spamFeatures = spam.map(lambda email: tf.transform(email.split()))

In [37]:
tramFeatures = ham.map(lambda email: [[1], [email.split()]])
tramFeatures = tramFeatures.toDF(schema=["labels", "words"])
v = tf.transform(tramFeatures)
v.show()

Py4JJavaError: An error occurred while calling o612.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 45.0 failed 1 times, most recent failure: Lost task 0.0 in stage 45.0 (TID 139, localhost): org.apache.spark.SparkException: HashingTF with murmur3 algorithm does not support type scala.collection.mutable.WrappedArray.ofRef of input data.
	at org.apache.spark.mllib.feature.HashingTF$.murmur3Hash(HashingTF.scala:164)
	at org.apache.spark.mllib.feature.HashingTF$$anonfun$getHashFunction$1.apply(HashingTF.scala:84)
	at org.apache.spark.mllib.feature.HashingTF$$anonfun$getHashFunction$1.apply(HashingTF.scala:84)
	at org.apache.spark.mllib.feature.HashingTF$$anonfun$transform$1.apply(HashingTF.scala:101)
	at org.apache.spark.mllib.feature.HashingTF$$anonfun$transform$1.apply(HashingTF.scala:100)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.WrappedArray.foreach(WrappedArray.sca

In [30]:
negativeClass.count()

Py4JError: An error occurred while calling o284.__getnewargs__. Trace:
py4j.Py4JException: Method __getnewargs__([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:272)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:128)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:211)
	at java.lang.Thread.run(Thread.java:745)



In [28]:
from pyspark.sql import Row

positiveClass = spamFeatures.map(lambda record: Row(label=1, features=record))
negativeClass = hamFeatures.map(lambda record: Row(label=0, features=record))
trainRDD = positiveClass.union(negativeClass)

<class 'pyspark.rdd.PipelinedRDD'>


Py4JError: An error occurred while calling o284.__getnewargs__. Trace:
py4j.Py4JException: Method __getnewargs__([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:272)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:128)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:211)
	at java.lang.Thread.run(Thread.java:745)



In [26]:
trainDF = sqlCtx.createDataFrame(trainRDD)

NameError: name 'trainRDD' is not defined

In [ ]:
print type(trainDF)

In [ ]:
trainDF.show()

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(trainDF)

In [ ]:
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))